# Cancer Dataset with PCA & SVM

In this exercise, we are again analyzing ultrasound data of breast cancer tumors. Several characteristics of the tumor are given in the dataset. First, we will be reducing the number of dimensions with PCA, then we will create a model that will predict whether or not a patient has a positive breast cancer diagnosis based off of the tumor characteristics.

## Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Get the Data

We'll use the built in breast cancer dataset from Scikit Learn. We can get with the load function:

In [3]:
from sklearn.datasets import load_breast_cancer

In [4]:
cancer = load_breast_cancer()

The data set is presented in a dictionary form:

In [5]:
cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

We can grab information and arrays out of this dictionary to set up our data frame and understanding of the features:

In [6]:
print(cancer['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [7]:
cancer['feature_names']

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

## Set up DataFrame

In [8]:
df_feat = pd.DataFrame(cancer['data'],columns=cancer['feature_names'])
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [9]:
cancer['target']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

# Exploratory Data Analysis



We'll skip the Data Viz part for this lecture since we already have looked at the data.

In [10]:
X = pd.DataFrame(cancer['data'],columns=cancer['feature_names'])

y= cancer["target"]

## Train Test Split

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

## Creating a Pipeline

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [14]:
pipe = Pipeline([('scaler', StandardScaler()),
                ('pca', PCA()),
                ('svm', SVC(random_state=123))])

We define param grid for Grid Search

In [15]:
param_grid = {'pca__n_components':[2,3,4,5], # choose the number of pca components
        'svm__kernel': ['linear', 'rbf'],
         'svm__C': [0.1,1, 10, 100, 1000],
         'svm__gamma': ["auto","scale"]}

In [16]:
grid = GridSearchCV(estimator=pipe,param_grid=param_grid,verbose=3)

In [17]:
grid.fit(X_train,y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV 1/5] END pca__n_components=2, svm__C=0.1, svm__gamma=auto, svm__kernel=linear;, score=0.925 total time=   0.0s
[CV 2/5] END pca__n_components=2, svm__C=0.1, svm__gamma=auto, svm__kernel=linear;, score=0.938 total time=   0.0s
[CV 3/5] END pca__n_components=2, svm__C=0.1, svm__gamma=auto, svm__kernel=linear;, score=0.975 total time=   0.0s
[CV 4/5] END pca__n_components=2, svm__C=0.1, svm__gamma=auto, svm__kernel=linear;, score=0.949 total time=   0.0s
[CV 5/5] END pca__n_components=2, svm__C=0.1, svm__gamma=auto, svm__kernel=linear;, score=0.962 total time=   0.0s
[CV 1/5] END pca__n_components=2, svm__C=0.1, svm__gamma=auto, svm__kernel=rbf;, score=0.900 total time=   0.0s
[CV 2/5] END pca__n_components=2, svm__C=0.1, svm__gamma=auto, svm__kernel=rbf;, score=0.925 total time=   0.0s
[CV 3/5] END pca__n_components=2, svm__C=0.1, svm__gamma=auto, svm__kernel=rbf;, score=0.950 total time=   0.0s
[CV 4/5] END pca__n_compone

GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('svm', SVC(random_state=123))]),
             param_grid={'pca__n_components': [2, 3, 4, 5],
                         'svm__C': [0.1, 1, 10, 100, 1000],
                         'svm__gamma': ['auto', 'scale'],
                         'svm__kernel': ['linear', 'rbf']},
             verbose=3)

In [18]:
grid.best_params_

{'pca__n_components': 5,
 'svm__C': 0.1,
 'svm__gamma': 'auto',
 'svm__kernel': 'linear'}

In [19]:
grid_predictions = grid.predict(X_test)

In [20]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,grid_predictions))

[[ 61   5]
 [  2 103]]


In [21]:
print(classification_report(y_test,grid_predictions))

              precision    recall  f1-score   support

           0       0.97      0.92      0.95        66
           1       0.95      0.98      0.97       105

    accuracy                           0.96       171
   macro avg       0.96      0.95      0.96       171
weighted avg       0.96      0.96      0.96       171

